In [1]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import torch.optim as optim
import torch.nn as nn
import torchmetrics
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [2]:
df = pd.read_csv("/kaggle/input/banana/banana_quality.csv")

In [3]:
df.head()

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,-1.924968,0.468078,3.077832,-1.472177,0.294799,2.435570,0.271290,Good
1,-2.409751,0.486870,0.346921,-2.495099,-0.892213,2.067549,0.307325,Good
2,-0.357607,1.483176,1.568452,-2.645145,-0.647267,3.090643,1.427322,Good
3,-0.868524,1.566201,1.889605,-1.273761,-1.006278,1.873001,0.477862,Good
4,0.651825,1.319199,-0.022459,-1.209709,-1.430692,1.078345,2.812442,Good


In [4]:
df["Quality"] = df["Quality"].replace({"Good":1, "Bad":0})

/tmp/ipykernel_36/3793629957.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Quality"] = df["Quality"].replace({"Good":1, "Bad":0})


In [9]:
x_train, x_test, y_train, y_test = train_test_split(df.drop("Quality", axis=1), df["Quality"])

In [10]:
x_train.shape, y_train.shape

((6000, 7), (6000,))

In [11]:
x_test.shape, y_test.shape

((2000, 7), (2000,))

In [15]:
model = nn.Sequential(
    nn.Linear(7, 26),
    nn.ReLU(),
    nn.Linear(26, 34),
    nn.ReLU(),
    nn.Linear(34, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
    nn.Sigmoid()
)

In [16]:
model

Sequential(
  (0): Linear(in_features=7, out_features=26, bias=True)
  (1): ReLU()
  (2): Linear(in_features=26, out_features=34, bias=True)
  (3): ReLU()
  (4): Linear(in_features=34, out_features=10, bias=True)
  (5): ReLU()
  (6): Linear(in_features=10, out_features=1, bias=True)
  (7): Sigmoid()
)

In [17]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [18]:
ds_train = TensorDataset(torch.Tensor(x_train.values).float(),
                  torch.Tensor(y_train.values).float())
dl_train = DataLoader(ds_train, batch_size=4, shuffle=True)

In [19]:
ds_test = TensorDataset(torch.Tensor(x_test.values).float(),
                  torch.Tensor(y_test.values).float())
dl_test = DataLoader(ds_test, batch_size=4, shuffle=True)

In [24]:
for i in range(10):
    total_loss = 0.0
    for batch in dl_train:
        features, qual = batch
        optimizer.zero_grad()
        output = model(features)
        loss = criterion(output, qual)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"loss of epoch {i+1}: {total_loss/len(dl_train)}")

loss of epoch 1: 0.2487911581993103
loss of epoch 2: 0.24833066761493683
loss of epoch 3: 0.2483600527048111
loss of epoch 4: 0.24853260815143585
loss of epoch 5: 0.24855485558509827
loss of epoch 6: 0.24827983975410461
loss of epoch 7: 0.24808010458946228
loss of epoch 8: 0.2477942854166031
loss of epoch 9: 0.24812719225883484
loss of epoch 10: 0.2481181025505066


In [25]:
model.eval()
with torch.no_grad():
    total_loss = 0.0
    for features, age in dl_test:
        output = model(features)
        loss = criterion(output, age)
        total_loss += loss
    print(f"loss of validation set: {total_loss/len(dl_test)}")

loss of validation set: 0.24854952096939087
